In [1]:
# The goal of this project is to create a model from the fifa data which is given:
#     - The first step is to clean the data 

In [2]:
# importing all the required libraries
import pandas as pd
import numpy as np
import math
from sklearn import linear_model

import seaborn as sns 
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib
import pickle 

In [3]:
# importing the data file
df= pd.read_csv("fifa21_validate.csv")
df

,ID,Name,Age,Nationality,Club,BP,Position,Team & Contract,Height,Weight,...,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,GK,OVA
0,219461,E. Palmer-Brown,23,United States,FK Austria Wien,CB,CB,"FK Austria Wien Jun 30, 2021 On Loan","6'2""",194lbs,...,61+2,61+2,59+2,61+2,67+2,67+2,67+2,61+2,16+2,67
1,221896,D. Avdijaj,22,Kosovo,Heart of Midlothian,CAM,LM CAM,Heart of Midlothian 2020 ~ 2020,"5'8""",154lbs,...,47+2,47+2,49+2,45+2,38+2,38+2,38+2,45+2,17+2,68
2,247428,D. Ochoa,19,United States,Real Salt Lake,GK,GK,Real Salt Lake 2018 ~ 2020,"6'2""",176lbs,...,18+2,18+2,16+2,16+2,18+2,18+2,18+2,16+2,53+2,54
3,255120,N. Kenneh,16,England,Leeds United,CDM,CB CDM RB,Leeds United 2020 ~ 2022,"6'3""",170lbs,...,54+2,54+2,53+2,53+2,54+2,54+2,54+2,53+2,11+2,55
4,215556,E. Fernandes,24,Switzerland,1. FSV Mainz 05,CDM,CM CDM,1. FSV Mainz 05 2019 ~ 2023,"6'2""",170lbs,...,72+2,72+2,70+2,69+2,68+2,68+2,68+2,69+2,18+2,70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1994,239074,S. Aw,21,Senegal,Gil Vicente FC,LB,LB,Gil Vicente FC 2020 ~ 2023,"5'8""",143lbs,...,52+2,52+2,58+2,58+2,54+2,54+2,54+2,58+2,15+2,60
1995,241223,S. Mogi,21,Japan,Cerezo Osaka,GK,GK,Cerezo Osaka 2017 ~ 2021,"6'5""",176lbs,...,23+2,23+2,20+2,20+2,22+2,22+2,22+2,20+2,58+2,59
1996,210930,Carles Gil,27,Spain,New England Revolution,RM,RM CAM CM,New England Revolution 2019 ~ 2024,"5'7""",146lbs,...,60+2,60+2,62+2,57+2,47+2,47+2,47+2,57+2,18+2,76
1997,162993,J. Perch,34,England,Mansfield Town,CDM,CDM RB CM,Mansfield Town 2020 ~ 2021,"5'11""",176lbs,...,62+1,62+1,60+2,60+2,63+0,63+0,63+0,60+2,14+2,63


# Data exploring and cleaning

In [4]:
# finding the information of the df to know the dtypes 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1999 entries, 0 to 1998
Columns: 101 entries, ID to OVA
dtypes: float64(10), int64(43), object(48)
memory usage: 1.5+ MB


In [5]:
# finding the column names
df.columns

Index(['ID', 'Name', 'Age', 'Nationality', 'Club', 'BP', 'Position',
       'Team & Contract', 'Height', 'Weight',
       ...
       'CDM', 'RDM', 'RWB', 'LB', 'LCB', 'CB', 'RCB', 'RB', 'GK', 'OVA'],
      dtype='object', length=101)

In [6]:
# finding the total count of null values in df
df.isna().sum()

ID             0
Name           0
Age            0
Nationality    0
Club           3
              ..
CB             0
RCB            0
RB             0
GK             0
OVA            0
Length: 101, dtype: int64

In [7]:
# to see the maximum number of columns 
pd.set_option('display.max_columns', None)

# to see the maximum number of columns 
# pd.set_option('display.max_rows', None)

In [8]:
# finding all the columns in the list format
columns = list(df.columns)
print(columns)

['ID', 'Name', 'Age', 'Nationality', 'Club', 'BP', 'Position', 'Team & Contract', 'Height', 'Weight', 'foot', 'Growth', 'Joined', 'Loan Date End', 'Value', 'Wage', 'Release Clause', 'Contract', 'Attacking', 'Crossing', 'Finishing', 'Heading Accuracy', 'Short Passing', 'Volleys', 'Skill', 'Dribbling', 'Curve', 'FK Accuracy', 'Long Passing', 'Ball Control', 'Movement', 'Acceleration', 'Sprint Speed', 'Agility', 'Reactions', 'Balance', 'Power', 'Shot Power', 'Jumping', 'Stamina', 'Strength', 'Long Shots', 'Mentality', 'Aggression', 'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure', 'Defending', 'Marking', 'Standing Tackle', 'Sliding Tackle', 'Goalkeeping', 'GK Diving', 'GK Handling', 'GK Kicking', 'GK Positioning', 'GK Reflexes', 'Total Stats', 'Base Stats', 'W/F', 'SM', 'A/W', 'D/W', 'IR', 'PAC', 'SHO', 'PAS', 'DRI', 'DEF', 'PHY', 'Hits', 'LS', 'ST', 'RS', 'LW', 'LF', 'CF', 'RF', 'RW', 'LAM', 'CAM', 'RAM', 'LM', 'LCM', 'CM', 'RCM', 'RM', 'LWB', 'LDM', 'CDM', 'RDM', 'RWB'

In [9]:
# finding shape to know how many rows and columns are present
df.shape

(1999, 101)

In [63]:
# Change headers names to lower case
df.columns= [col.lower() for col in df.columns]
# df.rename(columns={'st':'state'},inplace=True)
df.head()

,bp,foot,w/f,sm,a/w,d/w,ir,age,growth,value,wage,release clause,attacking,crossing,finishing,heading accuracy,short passing,volleys,skill,dribbling,curve,fk accuracy,long passing,ball control,movement,acceleration,sprint speed,agility,reactions,balance,power,shot power,jumping,stamina,strength,long shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing tackle,sliding tackle,goalkeeping,gk diving,gk handling,gk kicking,gk positioning,gk reflexes,total stats,base stats,pac,sho,pas,dri,def,phy,hits,ls,state,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,ova
0,CB,Right,2 ★,2★,Low,High,1 ★,23,7,975000.0,5000.0,0.0,230,47,21,62,60,40.0,228,44,43.0,36,51,54,303,60,68,63.0,63,49.0,288,48,77.0,51,87,25,246,68,62.0,38.0,39.0,39,49.0,200,68,67,65.0,56,11,8,15,13,9,1551,334,64,30,50,50,66,74,34.0,48.0,48.0,48.0,48.0,47.0,47.0,47.0,48.0,48.0,48.0,48.0,50.0,51.0,51.0,51.0,50.0,59.0,61.0,61.0,61.0,59.0,61.0,67.0,67.0,67.0,61.0,16.0,67
1,CAM,Right,4 ★,3★,High,Low,1 ★,22,5,1200000.0,3000.0,2200000.0,298,62,60,44,62,70.0,330,76,68.0,56,60,70,375,77,72,83.0,64,79.0,323,76,62.0,63,47,75,286,72,26.0,64.0,64.0,60,65.0,61,19,23,19.0,53,14,13,9,9,8,1726,358,74,67,62,74,24,57,12.0,64.0,64.0,64.0,68.0,68.0,68.0,68.0,68.0,68.0,68.0,68.0,67.0,61.0,61.0,61.0,67.0,49.0,47.0,47.0,47.0,49.0,45.0,38.0,38.0,38.0,45.0,17.0,68
2,GK,Right,2 ★,1★,Medium,Medium,1 ★,19,17,120000.0,500.0,249000.0,48,7,5,11,21,4.0,52,6,8.0,8,20,10,165,28,25,33.0,41,38.0,171,40,49.0,22,54,6,76,20,9.0,7.0,26.0,14,31.0,27,8,9,10.0,269,56,52,53,53,55,808,295,56,52,53,55,26,53,3.0,18.0,18.0,18.0,15.0,17.0,17.0,17.0,15.0,17.0,17.0,17.0,16.0,18.0,18.0,18.0,16.0,16.0,18.0,18.0,18.0,16.0,16.0,18.0,18.0,18.0,16.0,53.0,54
3,CDM,Right,3 ★,2★,Medium,Medium,1 ★,16,23,160000.0,500.0,464000.0,215,38,31,55,59,32.0,224,51,34.0,38,47,54,275,59,58,56.0,48,54.0,242,48,48.0,60,58,28,230,61,55.0,33.0,40.0,41,59.0,159,53,52,54.0,36,7,5,13,5,6,1381,303,58,34,47,52,53,59,6.0,46.0,46.0,46.0,47.0,46.0,46.0,46.0,47.0,47.0,47.0,47.0,49.0,49.0,49.0,49.0,49.0,53.0,54.0,54.0,54.0,53.0,53.0,54.0,54.0,54.0,53.0,11.0,55
4,CDM,Right,4 ★,2★,Medium,Medium,1 ★,24,5,2300000.0,13000.0,4300000.0,295,57,59,45,78,56.0,327,71,57.0,51,74,74,320,68,66,66.0,64,56.0,337,73,56.0,74,72,62,314,66,78.0,53.0,62.0,55,63.0,211,72,68,71.0,60,12,7,13,15,13,1864,407,67,62,68,70,69,71,45.0,63.0,63.0,63.0,66.0,66.0,66.0,66.0,66.0,68.0,68.0,68.0,67.0,70.0,70.0,70.0,67.0,70.0,72.0,72.0,72.0,70.0,69.0,68.0,68.0,68.0,69.0,18.0,70


In [11]:
#Categorical Features- extracting the columns which are object dtypes
df_c=df.select_dtypes(include = np.object)
df_c.tail()

,name,nationality,club,bp,position,team & contract,height,weight,foot,joined,loan date end,value,wage,release clause,contract,w/f,sm,a/w,d/w,ir,hits,ls,state,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
1994,S. Aw,Senegal,Gil Vicente FC,LB,LB,Gil Vicente FC 2020 ~ 2023,"5'8""",143lbs,Left,"Aug 9, 2020",NaN,€325K,€1K,€731K,2020 ~ 2023,3 ★,2★,High,Medium,1 ★,4,44+2,44+2,44+2,51+0,48+0,48+0,48+0,51+0,48+2,48+2,48+2,53+2,48+2,48+2,48+2,53+2,58+2,52+2,52+2,52+2,58+2,58+2,54+2,54+2,54+2,58+2,15+2
1995,S. Mogi,Japan,Cerezo Osaka,GK,GK,Cerezo Osaka 2017 ~ 2021,"6'5""",176lbs,Right,"Jan 1, 2017",NaN,€190K,€700,€285K,2017 ~ 2021,2 ★,1★,Medium,Medium,1 ★,3,22+2,22+2,22+2,20+0,22+0,22+0,22+0,20+0,23+2,23+2,23+2,22+2,24+2,24+2,24+2,22+2,20+2,23+2,23+2,23+2,20+2,20+2,22+2,22+2,22+2,20+2,58+2
1996,Carles Gil,Spain,New England Revolution,RM,RM CAM CM,New England Revolution 2019 ~ 2024,"5'7""",146lbs,Left,"Feb 13, 2019",NaN,€8M,€9K,€12M,2019 ~ 2024,4 ★,4★,High,Medium,2 ★,15,67+2,67+2,67+2,74+0,73+0,73+0,73+0,74+0,76+0,76+0,76+0,75+1,74+2,74+2,74+2,75+1,62+2,60+2,60+2,60+2,62+2,57+2,47+2,47+2,47+2,57+2,18+2
1997,J. Perch,England,Mansfield Town,CDM,CDM RB CM,Mansfield Town 2020 ~ 2021,"5'11""",176lbs,Right,"Aug 13, 2020",NaN,€140K,€4K,€245K,2020 ~ 2021,3 ★,2★,Medium,Medium,1 ★,4,55+2,55+2,55+2,56+0,55+0,55+0,55+0,56+0,57+2,57+2,57+2,57+2,59+2,59+2,59+2,57+2,60+2,62+1,62+1,62+1,60+2,60+2,63+0,63+0,63+0,60+2,14+2
1998,A. Oeynhausen,Germany,SC Paderborn 07,CAM,CM,SC Paderborn 07 2020 ~ 2022,"5'10""",150lbs,Right,"Jan 1, 2020",NaN,€425K,€1K,€946K,2020 ~ 2022,3 ★,2★,Medium,Medium,1 ★,5,57+2,57+2,57+2,59+0,59+0,59+0,59+0,59+0,61+2,61+2,61+2,60+2,60+2,60+2,60+2,60+2,55+2,55+2,55+2,55+2,55+2,54+2,51+2,51+2,51+2,54+2,16+2


In [12]:
#numerical Features- extracting the columns which are numerical dtypes
df_n=df.select_dtypes(include = np.number)
df_n.head()

,id,age,growth,attacking,crossing,finishing,heading accuracy,short passing,volleys,skill,dribbling,curve,fk accuracy,long passing,ball control,movement,acceleration,sprint speed,agility,reactions,balance,power,shot power,jumping,stamina,strength,long shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing tackle,sliding tackle,goalkeeping,gk diving,gk handling,gk kicking,gk positioning,gk reflexes,total stats,base stats,pac,sho,pas,dri,def,phy,ova
0,219461,23,7,230,47,21,62,60,40.0,228,44,43.0,36,51,54,303,60,68,63.0,63,49.0,288,48,77.0,51,87,25,246,68,62.0,38.0,39.0,39,49.0,200,68,67,65.0,56,11,8,15,13,9,1551,334,64,30,50,50,66,74,67
1,221896,22,5,298,62,60,44,62,70.0,330,76,68.0,56,60,70,375,77,72,83.0,64,79.0,323,76,62.0,63,47,75,286,72,26.0,64.0,64.0,60,65.0,61,19,23,19.0,53,14,13,9,9,8,1726,358,74,67,62,74,24,57,68
2,247428,19,17,48,7,5,11,21,4.0,52,6,8.0,8,20,10,165,28,25,33.0,41,38.0,171,40,49.0,22,54,6,76,20,9.0,7.0,26.0,14,31.0,27,8,9,10.0,269,56,52,53,53,55,808,295,56,52,53,55,26,53,54
3,255120,16,23,215,38,31,55,59,32.0,224,51,34.0,38,47,54,275,59,58,56.0,48,54.0,242,48,48.0,60,58,28,230,61,55.0,33.0,40.0,41,59.0,159,53,52,54.0,36,7,5,13,5,6,1381,303,58,34,47,52,53,59,55
4,215556,24,5,295,57,59,45,78,56.0,327,71,57.0,51,74,74,320,68,66,66.0,64,56.0,337,73,56.0,74,72,62,314,66,78.0,53.0,62.0,55,63.0,211,72,68,71.0,60,12,7,13,15,13,1864,407,67,62,68,70,69,71,70


In [13]:
# finding null values through out numerical df
df_n.isna().sum()

id                   0
age                  0
growth               0
attacking            0
crossing             0
finishing            0
heading accuracy     0
short passing        0
volleys              3
skill                0
dribbling            0
curve                3
fk accuracy          0
long passing         0
ball control         0
movement             0
acceleration         0
sprint speed         0
agility              3
reactions            0
balance              3
power                0
shot power           0
jumping              3
stamina              0
strength             0
long shots           0
mentality            0
aggression           0
interceptions        0
positioning          0
vision               3
penalties            0
composure           50
defending            0
marking              0
standing tackle      0
sliding tackle       3
goalkeeping          0
gk diving            0
gk handling          0
gk kicking           0
gk positioning       0
gk reflexes

In [14]:
# def show_categorical_values(df):
#     # Get the list of categorical columns
#     categorical_columns = df.select_dtypes(include='object').columns

#     # Print unique values for each categorical column
#     for column in categorical_columns:
#         print(f"Unique values for column '{column}':")
#         print(df[column].unique())
#         print("\n")

In [15]:
# finding the unique values using .value_counts()
#- here it looks this column has lot of unique values 
# it is not preferred for encoding when there are many unique values                                                 
df_c["nationality"].value_counts()

England       180
Spain         145
Germany       127
France        119
Brazil        114
             ... 
Eritrea         1
Thailand        1
Zambia          1
Mauritania      1
Kazakhstan      1
Name: nationality, Length: 112, dtype: int64

In [16]:
# to know the categorical data 
df_c.head()

,name,nationality,club,bp,position,team & contract,height,weight,foot,joined,loan date end,value,wage,release clause,contract,w/f,sm,a/w,d/w,ir,hits,ls,state,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
0,E. Palmer-Brown,United States,FK Austria Wien,CB,CB,"FK Austria Wien Jun 30, 2021 On Loan","6'2""",194lbs,Right,"Feb 8, 2018","Jun 30, 2021",€975K,€5K,€0,"Jun 30, 2021 On Loan",2 ★,2★,Low,High,1 ★,34,48+2,48+2,48+2,48+0,47+0,47+0,47+0,48+0,48+2,48+2,48+2,50+2,51+2,51+2,51+2,50+2,59+2,61+2,61+2,61+2,59+2,61+2,67+2,67+2,67+2,61+2,16+2
1,D. Avdijaj,Kosovo,Heart of Midlothian,CAM,LM CAM,Heart of Midlothian 2020 ~ 2020,"5'8""",154lbs,Right,"Jan 20, 2020",NaN,€1.2M,€3K,€2.2M,2020 ~ 2020,4 ★,3★,High,Low,1 ★,12,64+2,64+2,64+2,68+0,68+0,68+0,68+0,68+0,68+2,68+2,68+2,67+2,61+2,61+2,61+2,67+2,49+2,47+2,47+2,47+2,49+2,45+2,38+2,38+2,38+2,45+2,17+2
2,D. Ochoa,United States,Real Salt Lake,GK,GK,Real Salt Lake 2018 ~ 2020,"6'2""",176lbs,Right,"Nov 28, 2018",NaN,€120K,€500,€249K,2018 ~ 2020,2 ★,1★,Medium,Medium,1 ★,3,18+2,18+2,18+2,15+0,17+0,17+0,17+0,15+0,17+2,17+2,17+2,16+2,18+2,18+2,18+2,16+2,16+2,18+2,18+2,18+2,16+2,16+2,18+2,18+2,18+2,16+2,53+2
3,N. Kenneh,England,Leeds United,CDM,CB CDM RB,Leeds United 2020 ~ 2022,"6'3""",170lbs,Right,"Jan 10, 2020",NaN,€160K,€500,€464K,2020 ~ 2022,3 ★,2★,Medium,Medium,1 ★,6,46+2,46+2,46+2,47+0,46+0,46+0,46+0,47+0,47+2,47+2,47+2,49+2,49+2,49+2,49+2,49+2,53+2,54+2,54+2,54+2,53+2,53+2,54+2,54+2,54+2,53+2,11+2
4,E. Fernandes,Switzerland,1. FSV Mainz 05,CDM,CM CDM,1. FSV Mainz 05 2019 ~ 2023,"6'2""",170lbs,Right,"Jul 1, 2019",NaN,€2.3M,€13K,€4.3M,2019 ~ 2023,4 ★,2★,Medium,Medium,1 ★,45,63+2,63+2,63+2,66+0,66+0,66+0,66+0,66+0,68+2,68+2,68+2,67+2,70+2,70+2,70+2,67+2,70+2,72+2,72+2,72+2,70+2,69+2,68+2,68+2,68+2,69+2,18+2


In [17]:
# finding the sum of duplicates in the df which is not much important according to this data or the output shown
duplicates_count = df.apply(lambda column: column.duplicated().sum())
# duplicates_count["name"]
duplicates_count

id                0
name             24
age            1973
nationality    1887
club           1300
               ... 
cb             1804
rcb            1804
rb             1829
gk             1894
ova            1955
Length: 101, dtype: int64

In [18]:
# finding the total nan values in each column from the df
nan_count = df.isnull().sum()

# Create a new DataFrame with NaN count
nan_count_df = pd.DataFrame({'Column': nan_count.index, 'NaN Count': nan_count.values})

# print("NaN count in each column:")
nan_count_df.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
Column,id,name,age,nationality,club,bp,position,team & contract,height,weight,foot,growth,joined,loan date end,value,wage,release clause,contract,attacking,crossing,finishing,heading accuracy,short passing,volleys,skill,dribbling,curve,fk accuracy,long passing,ball control,movement,acceleration,sprint speed,agility,reactions,balance,power,shot power,jumping,stamina,strength,long shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing tackle,sliding tackle,goalkeeping,gk diving,gk handling,gk kicking,gk positioning,gk reflexes,total stats,base stats,w/f,sm,a/w,d/w,ir,pac,sho,pas,dri,def,phy,hits,ls,state,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,ova
NaN Count,0,0,0,0,3,0,55,0,0,0,0,0,3,1889,0,0,0,0,0,0,0,0,0,3,0,0,3,0,0,0,0,0,0,3,0,3,0,0,3,0,0,0,0,0,0,0,3,0,50,0,0,0,3,0,0,0,0,0,0,0,0,0,0,8,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [19]:
# along with nan values finding the percentage of nan values in df
# this step is not important for this task

# Calculate percentage of NaN values in each column
total_rows = len(df)
nan_percentage = (nan_count / total_rows) * 100

# Create a new DataFrame to display the counts and percentages
nan_summary = pd.DataFrame({
    'NaN Count': nan_count,
    'NaN Percentage': nan_percentage
})

print("NaN Count and Percentage in each column:")
nan_summary.T

NaN Count and Percentage in each column:


,id,name,age,nationality,club,bp,position,team & contract,height,weight,foot,growth,joined,loan date end,value,wage,release clause,contract,attacking,crossing,finishing,heading accuracy,short passing,volleys,skill,dribbling,curve,fk accuracy,long passing,ball control,movement,acceleration,sprint speed,agility,reactions,balance,power,shot power,jumping,stamina,strength,long shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing tackle,sliding tackle,goalkeeping,gk diving,gk handling,gk kicking,gk positioning,gk reflexes,total stats,base stats,w/f,sm,a/w,d/w,ir,pac,sho,pas,dri,def,phy,hits,ls,state,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,ova
NaN Count,0.0,0.0,0.0,0.0,3.000000,0.0,55.000000,0.0,0.0,0.0,0.0,0.0,3.000000,1889.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.000000,0.0,0.0,3.000000,0.0,0.0,0.0,0.0,0.0,0.0,3.000000,0.0,3.000000,0.0,0.0,3.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.000000,0.0,50.000000,0.0,0.0,0.0,3.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0000,8.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
NaN Percentage,0.0,0.0,0.0,0.0,0.150075,0.0,2.751376,0.0,0.0,0.0,0.0,0.0,0.150075,94.497249,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.150075,0.0,0.0,0.150075,0.0,0.0,0.0,0.0,0.0,0.0,0.150075,0.0,0.150075,0.0,0.0,0.150075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.150075,0.0,2.501251,0.0,0.0,0.0,0.150075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.4002,0.4002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Data transformation

In [20]:
def convert_currency(currency_str):
    if currency_str.endswith('K'):
        return float(currency_str[1:-1]) * 1000
    elif currency_str.endswith('M'):
        return float(currency_str[1:-1]) * 1000000
    else:
        return float(currency_str[1:])
# Apply the conversion function to the 'Salary' column
df['value'] = df['value'].apply(convert_currency)
df['wage'] = df['wage'].apply(convert_currency)
df['release clause'] = df['release clause'].apply(convert_currency)


df.tail()

,id,name,age,nationality,club,bp,position,team & contract,height,weight,foot,growth,joined,loan date end,value,wage,release clause,contract,attacking,crossing,finishing,heading accuracy,short passing,volleys,skill,dribbling,curve,fk accuracy,long passing,ball control,movement,acceleration,sprint speed,agility,reactions,balance,power,shot power,jumping,stamina,strength,long shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing tackle,sliding tackle,goalkeeping,gk diving,gk handling,gk kicking,gk positioning,gk reflexes,total stats,base stats,w/f,sm,a/w,d/w,ir,pac,sho,pas,dri,def,phy,hits,ls,state,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,ova
1994,239074,S. Aw,21,Senegal,Gil Vicente FC,LB,LB,Gil Vicente FC 2020 ~ 2023,"5'8""",143lbs,Left,11,"Aug 9, 2020",NaN,325000.0,1000.0,731000.0,2020 ~ 2023,197,59,23,42,51,22.0,207,51,30.0,29,41,56,344,74,78,63.0,50,79.0,245,31,66.0,68,50,30,232,59,52.0,51.0,34.0,36,59.0,167,52,57,58.0,51,6,15,7,10,13,1443,315,3 ★,2★,High,Medium,1 ★,76,28,46,55,53,57,4,44+2,44+2,44+2,51+0,48+0,48+0,48+0,51+0,48+2,48+2,48+2,53+2,48+2,48+2,48+2,53+2,58+2,52+2,52+2,52+2,58+2,58+2,54+2,54+2,54+2,58+2,15+2,60
1995,241223,S. Mogi,21,Japan,Cerezo Osaka,GK,GK,Cerezo Osaka 2017 ~ 2021,"6'5""",176lbs,Right,9,"Jan 1, 2017",NaN,190000.0,700.0,285000.0,2017 ~ 2021,77,13,9,14,34,7.0,70,9,13.0,10,25,13,192,31,30,36.0,59,36.0,177,43,44.0,23,60,7,88,21,8.0,4.0,36.0,19,33.0,36,14,11,11.0,288,60,55,57,54,62,928,318,2 ★,1★,Medium,Medium,1 ★,60,55,57,62,30,54,3,22+2,22+2,22+2,20+0,22+0,22+0,22+0,20+0,23+2,23+2,23+2,22+2,24+2,24+2,24+2,22+2,20+2,23+2,23+2,23+2,20+2,20+2,22+2,22+2,22+2,20+2,58+2,59
1996,210930,Carles Gil,27,Spain,New England Revolution,RM,RM CAM CM,New England Revolution 2019 ~ 2024,"5'7""",146lbs,Left,0,"Feb 13, 2019",NaN,8000000.0,9000.0,12000000.0,2019 ~ 2024,332,76,72,34,79,71.0,373,77,76.0,65,77,78,351,64,65,74.0,73,75.0,329,63,60.0,83,57,66,306,41,40.0,75.0,81.0,69,75.0,118,36,43,39.0,58,12,15,16,9,6,1867,388,4 ★,4★,High,Medium,2 ★,65,69,78,77,39,60,15,67+2,67+2,67+2,74+0,73+0,73+0,73+0,74+0,76+0,76+0,76+0,75+1,74+2,74+2,74+2,75+1,62+2,60+2,60+2,60+2,62+2,57+2,47+2,47+2,47+2,57+2,18+2,76
1997,162993,J. Perch,34,England,Mansfield Town,CDM,CDM RB CM,Mansfield Town 2020 ~ 2021,"5'11""",176lbs,Right,0,"Aug 13, 2020",NaN,140000.0,4000.0,245000.0,2020 ~ 2021,268,58,44,61,62,43.0,261,56,47.0,37,60,61,288,54,52,57.0,61,64.0,300,53,65.0,68,68,46,297,74,62.0,54.0,56.0,51,60.0,182,61,62,59.0,43,10,7,9,9,8,1639,346,3 ★,2★,Medium,Medium,1 ★,53,47,58,58,61,69,4,55+2,55+2,55+2,56+0,55+0,55+0,55+0,56+0,57+2,57+2,57+2,57+2,59+2,59+2,59+2,57+2,60+2,62+1,62+1,62+1,60+2,60+2,63+0,63+0,63+0,60+2,14+2,63
1998,254882,A. Oeynhausen,18,Germany,SC Paderborn 07,CAM,CM,SC Paderborn 07 2020 ~ 2022,"5'10""",150lbs,Right,16,"Jan 1, 2020",NaN,425000.0,1000.0,946000.0,2020 ~ 2022,276,49,54,51,70,52.0,300,60,61.0,51,66,62,326,68,61,69.0,55,73.0,256,65,50.0,52,45,44,260,45,38.0,51.0,63.0,63,65.0,159,48,57,54.0,57,8,15,11,11,12,1634,339,3 ★,2★,Medium,Medium,1 ★,64,54,62,62,50,47,5,57+2,57+2,57+2,59+0,59+0,59+0,59+0,59+0,61+2,61+2,61+2,60+2,60+2,60+2,60+2,60+2,55+2,55+2,55+2,55+2,55+2,54+2,51+2,51+2,51+2,54+2,16+2,60


In [21]:
# drop team & contract,value, wage andrelease clause from df
df=df.drop(['team & contract'], axis=1)

df

,id,name,age,nationality,club,bp,position,height,weight,foot,growth,joined,loan date end,value,wage,release clause,contract,attacking,crossing,finishing,heading accuracy,short passing,volleys,skill,dribbling,curve,fk accuracy,long passing,ball control,movement,acceleration,sprint speed,agility,reactions,balance,power,shot power,jumping,stamina,strength,long shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing tackle,sliding tackle,goalkeeping,gk diving,gk handling,gk kicking,gk positioning,gk reflexes,total stats,base stats,w/f,sm,a/w,d/w,ir,pac,sho,pas,dri,def,phy,hits,ls,state,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,ova
0,219461,E. Palmer-Brown,23,United States,FK Austria Wien,CB,CB,"6'2""",194lbs,Right,7,"Feb 8, 2018","Jun 30, 2021",975000.0,5000.0,0.0,"Jun 30, 2021 On Loan",230,47,21,62,60,40.0,228,44,43.0,36,51,54,303,60,68,63.0,63,49.0,288,48,77.0,51,87,25,246,68,62.0,38.0,39.0,39,49.0,200,68,67,65.0,56,11,8,15,13,9,1551,334,2 ★,2★,Low,High,1 ★,64,30,50,50,66,74,34,48+2,48+2,48+2,48+0,47+0,47+0,47+0,48+0,48+2,48+2,48+2,50+2,51+2,51+2,51+2,50+2,59+2,61+2,61+2,61+2,59+2,61+2,67+2,67+2,67+2,61+2,16+2,67
1,221896,D. Avdijaj,22,Kosovo,Heart of Midlothian,CAM,LM CAM,"5'8""",154lbs,Right,5,"Jan 20, 2020",NaN,1200000.0,3000.0,2200000.0,2020 ~ 2020,298,62,60,44,62,70.0,330,76,68.0,56,60,70,375,77,72,83.0,64,79.0,323,76,62.0,63,47,75,286,72,26.0,64.0,64.0,60,65.0,61,19,23,19.0,53,14,13,9,9,8,1726,358,4 ★,3★,High,Low,1 ★,74,67,62,74,24,57,12,64+2,64+2,64+2,68+0,68+0,68+0,68+0,68+0,68+2,68+2,68+2,67+2,61+2,61+2,61+2,67+2,49+2,47+2,47+2,47+2,49+2,45+2,38+2,38+2,38+2,45+2,17+2,68
2,247428,D. Ochoa,19,United States,Real Salt Lake,GK,GK,"6'2""",176lbs,Right,17,"Nov 28, 2018",NaN,120000.0,500.0,249000.0,2018 ~ 2020,48,7,5,11,21,4.0,52,6,8.0,8,20,10,165,28,25,33.0,41,38.0,171,40,49.0,22,54,6,76,20,9.0,7.0,26.0,14,31.0,27,8,9,10.0,269,56,52,53,53,55,808,295,2 ★,1★,Medium,Medium,1 ★,56,52,53,55,26,53,3,18+2,18+2,18+2,15+0,17+0,17+0,17+0,15+0,17+2,17+2,17+2,16+2,18+2,18+2,18+2,16+2,16+2,18+2,18+2,18+2,16+2,16+2,18+2,18+2,18+2,16+2,53+2,54
3,255120,N. Kenneh,16,England,Leeds United,CDM,CB CDM RB,"6'3""",170lbs,Right,23,"Jan 10, 2020",NaN,160000.0,500.0,464000.0,2020 ~ 2022,215,38,31,55,59,32.0,224,51,34.0,38,47,54,275,59,58,56.0,48,54.0,242,48,48.0,60,58,28,230,61,55.0,33.0,40.0,41,59.0,159,53,52,54.0,36,7,5,13,5,6,1381,303,3 ★,2★,Medium,Medium,1 ★,58,34,47,52,53,59,6,46+2,46+2,46+2,47+0,46+0,46+0,46+0,47+0,47+2,47+2,47+2,49+2,49+2,49+2,49+2,49+2,53+2,54+2,54+2,54+2,53+2,53+2,54+2,54+2,54+2,53+2,11+2,55
4,215556,E. Fernandes,24,Switzerland,1. FSV Mainz 05,CDM,CM CDM,"6'2""",170lbs,Right,5,"Jul 1, 2019",NaN,2300000.0,13000.0,4300000.0,2019 ~ 2023,295,57,59,45,78,56.0,327,71,57.0,51,74,74,320,68,66,66.0,64,56.0,337,73,56.0,74,72,62,314,66,78.0,53.0,62.0,55,63.0,211,72,68,71.0,60,12,7,13,15,13,1864,407,4 ★,2★,Medium,Medium,1 ★,67,62,68,70,69,71,45,63+2,63+2,63+2,66+0,66+0,66+0,66+0,66+0,68+2,68+2,68+2,67+2,70+2,70+2,70+2,67+2,70+2,72+2,72+2,72+2,70+2,69+2,68+2,68+2,68+2,69+2,18+2,70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1994,239074,S. Aw,21,Senegal,Gil Vicente FC,LB,LB,"5'8""",143lbs,Left,11,"Aug 9, 2020",NaN,325000.0,1000.0,731000.0,2020 ~ 2023,197,59,23,42,51,22.0,207,51,30.0,29,41,56,344,74,78,63.0,50,79.0,245,31,66.0,68,50,30,232,59,52.0,51.0,34.0,36,59.0,167,52,57,58.0,51,6,15,7,10,13,1443,315,3 ★,2★,High,Medium,1 ★,76,28,46,55,53,57,4,44+2,44+2,44+2,51+0,48+0,48+0,48+0,51+0,48+2,48+2,48+2,53+2,48+2,48+2,48+2,53+2,58+2,52+2,52+2,52+2,58+2,58+2,54+2,54+2,54+2,58+2,15+2,60
1995,241223

In [22]:
# converting object into int datatype
df['hits'] = pd.to_numeric(df['hits'], errors='coerce', downcast='integer')
df.head()

,id,name,age,nationality,club,bp,position,height,weight,foot,growth,joined,loan date end,value,wage,release clause,contract,attacking,crossing,finishing,heading accuracy,short passing,volleys,skill,dribbling,curve,fk accuracy,long passing,ball control,movement,acceleration,sprint speed,agility,reactions,balance,power,shot power,jumping,stamina,strength,long shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing tackle,sliding tackle,goalkeeping,gk diving,gk handling,gk kicking,gk positioning,gk reflexes,total stats,base stats,w/f,sm,a/w,d/w,ir,pac,sho,pas,dri,def,phy,hits,ls,state,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,ova
0,219461,E. Palmer-Brown,23,United States,FK Austria Wien,CB,CB,"6'2""",194lbs,Right,7,"Feb 8, 2018","Jun 30, 2021",975000.0,5000.0,0.0,"Jun 30, 2021 On Loan",230,47,21,62,60,40.0,228,44,43.0,36,51,54,303,60,68,63.0,63,49.0,288,48,77.0,51,87,25,246,68,62.0,38.0,39.0,39,49.0,200,68,67,65.0,56,11,8,15,13,9,1551,334,2 ★,2★,Low,High,1 ★,64,30,50,50,66,74,34.0,48+2,48+2,48+2,48+0,47+0,47+0,47+0,48+0,48+2,48+2,48+2,50+2,51+2,51+2,51+2,50+2,59+2,61+2,61+2,61+2,59+2,61+2,67+2,67+2,67+2,61+2,16+2,67
1,221896,D. Avdijaj,22,Kosovo,Heart of Midlothian,CAM,LM CAM,"5'8""",154lbs,Right,5,"Jan 20, 2020",NaN,1200000.0,3000.0,2200000.0,2020 ~ 2020,298,62,60,44,62,70.0,330,76,68.0,56,60,70,375,77,72,83.0,64,79.0,323,76,62.0,63,47,75,286,72,26.0,64.0,64.0,60,65.0,61,19,23,19.0,53,14,13,9,9,8,1726,358,4 ★,3★,High,Low,1 ★,74,67,62,74,24,57,12.0,64+2,64+2,64+2,68+0,68+0,68+0,68+0,68+0,68+2,68+2,68+2,67+2,61+2,61+2,61+2,67+2,49+2,47+2,47+2,47+2,49+2,45+2,38+2,38+2,38+2,45+2,17+2,68
2,247428,D. Ochoa,19,United States,Real Salt Lake,GK,GK,"6'2""",176lbs,Right,17,"Nov 28, 2018",NaN,120000.0,500.0,249000.0,2018 ~ 2020,48,7,5,11,21,4.0,52,6,8.0,8,20,10,165,28,25,33.0,41,38.0,171,40,49.0,22,54,6,76,20,9.0,7.0,26.0,14,31.0,27,8,9,10.0,269,56,52,53,53,55,808,295,2 ★,1★,Medium,Medium,1 ★,56,52,53,55,26,53,3.0,18+2,18+2,18+2,15+0,17+0,17+0,17+0,15+0,17+2,17+2,17+2,16+2,18+2,18+2,18+2,16+2,16+2,18+2,18+2,18+2,16+2,16+2,18+2,18+2,18+2,16+2,53+2,54
3,255120,N. Kenneh,16,England,Leeds United,CDM,CB CDM RB,"6'3""",170lbs,Right,23,"Jan 10, 2020",NaN,160000.0,500.0,464000.0,2020 ~ 2022,215,38,31,55,59,32.0,224,51,34.0,38,47,54,275,59,58,56.0,48,54.0,242,48,48.0,60,58,28,230,61,55.0,33.0,40.0,41,59.0,159,53,52,54.0,36,7,5,13,5,6,1381,303,3 ★,2★,Medium,Medium,1 ★,58,34,47,52,53,59,6.0,46+2,46+2,46+2,47+0,46+0,46+0,46+0,47+0,47+2,47+2,47+2,49+2,49+2,49+2,49+2,49+2,53+2,54+2,54+2,54+2,53+2,53+2,54+2,54+2,54+2,53+2,11+2,55
4,215556,E. Fernandes,24,Switzerland,1. FSV Mainz 05,CDM,CM CDM,"6'2""",170lbs,Right,5,"Jul 1, 2019",NaN,2300000.0,13000.0,4300000.0,2019 ~ 2023,295,57,59,45,78,56.0,327,71,57.0,51,74,74,320,68,66,66.0,64,56.0,337,73,56.0,74,72,62,314,66,78.0,53.0,62.0,55,63.0,211,72,68,71.0,60,12,7,13,15,13,1864,407,4 ★,2★,Medium,Medium,1 ★,67,62,68,70,69,71,45.0,63+2,63+2,63+2,66+0,66+0,66+0,66+0,66+0,68+2,68+2,68+2,67+2,70+2,70+2,70+2,67+2,70+2,72+2,72+2,72+2,70+2,69+2,68+2,68+2,68+2,69+2,18+2,70


In [23]:

# Split the column and remove the '+' with the values after them
columns = ['ls', 'state', 'rs', 'lw', 'lf', 'cf', 'rf', 'rw', 'lam', 'cam', 'ram', 'lm', 'lcm', 'cm', 'rcm',
           'rm', 'lwb', 'ldm', 'cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb', 'rcb', 'rb','gk']
# df[columns]

for colmun in columns: # for itertion of columns
    df[colmun] = df[colmun].str.split('+',expand=True)[0] # Split of string
    df[colmun] = df[colmun].str.split('-',expand=True)[0]
df.head(1)

,id,name,age,nationality,club,bp,position,height,weight,foot,growth,joined,loan date end,value,wage,release clause,contract,attacking,crossing,finishing,heading accuracy,short passing,volleys,skill,dribbling,curve,fk accuracy,long passing,ball control,movement,acceleration,sprint speed,agility,reactions,balance,power,shot power,jumping,stamina,strength,long shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing tackle,sliding tackle,goalkeeping,gk diving,gk handling,gk kicking,gk positioning,gk reflexes,total stats,base stats,w/f,sm,a/w,d/w,ir,pac,sho,pas,dri,def,phy,hits,ls,state,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,ova
0,219461,E. Palmer-Brown,23,United States,FK Austria Wien,CB,CB,"6'2""",194lbs,Right,7,"Feb 8, 2018","Jun 30, 2021",975000.0,5000.0,0.0,"Jun 30, 2021 On Loan",230,47,21,62,60,40.0,228,44,43.0,36,51,54,303,60,68,63.0,63,49.0,288,48,77.0,51,87,25,246,68,62.0,38.0,39.0,39,49.0,200,68,67,65.0,56,11,8,15,13,9,1551,334,2 ★,2★,Low,High,1 ★,64,30,50,50,66,74,34.0,48,48,48,48,47,47,47,48,48,48,48,50,51,51,51,50,59,61,61,61,59,61,67,67,67,61,16,67


In [24]:
# converting all the +2,+3 columns into numeric
for column in columns:
    df[column] = pd.to_numeric(df[column], errors='coerce', downcast='float')
df.head(1)

,id,name,age,nationality,club,bp,position,height,weight,foot,growth,joined,loan date end,value,wage,release clause,contract,attacking,crossing,finishing,heading accuracy,short passing,volleys,skill,dribbling,curve,fk accuracy,long passing,ball control,movement,acceleration,sprint speed,agility,reactions,balance,power,shot power,jumping,stamina,strength,long shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing tackle,sliding tackle,goalkeeping,gk diving,gk handling,gk kicking,gk positioning,gk reflexes,total stats,base stats,w/f,sm,a/w,d/w,ir,pac,sho,pas,dri,def,phy,hits,ls,state,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,ova
0,219461,E. Palmer-Brown,23,United States,FK Austria Wien,CB,CB,"6'2""",194lbs,Right,7,"Feb 8, 2018","Jun 30, 2021",975000.0,5000.0,0.0,"Jun 30, 2021 On Loan",230,47,21,62,60,40.0,228,44,43.0,36,51,54,303,60,68,63.0,63,49.0,288,48,77.0,51,87,25,246,68,62.0,38.0,39.0,39,49.0,200,68,67,65.0,56,11,8,15,13,9,1551,334,2 ★,2★,Low,High,1 ★,64,30,50,50,66,74,34.0,48.0,48.0,48.0,48.0,47.0,47.0,47.0,48.0,48.0,48.0,48.0,50.0,51.0,51.0,51.0,50.0,59.0,61.0,61.0,61.0,59.0,61.0,67.0,67.0,67.0,61.0,16.0,67


In [25]:
df_c=df.select_dtypes(include = np.object)
df_c.head(1)

,name,nationality,club,bp,position,height,weight,foot,joined,loan date end,contract,w/f,sm,a/w,d/w,ir
0,E. Palmer-Brown,United States,FK Austria Wien,CB,CB,"6'2""",194lbs,Right,"Feb 8, 2018","Jun 30, 2021","Jun 30, 2021 On Loan",2 ★,2★,Low,High,1 ★


In [26]:
# finding mode and filling mode value with nan values
column_modes = df_c.mode().iloc[0]

df_c = df_c.fillna(column_modes)
df_c.head(1)

,name,nationality,club,bp,position,height,weight,foot,joined,loan date end,contract,w/f,sm,a/w,d/w,ir
0,E. Palmer-Brown,United States,FK Austria Wien,CB,CB,"6'2""",194lbs,Right,"Feb 8, 2018","Jun 30, 2021","Jun 30, 2021 On Loan",2 ★,2★,Low,High,1 ★


In [27]:
# columns_to_delete = ['name', 'nationality','club','bp','position','joined','loan date end','contract']
columns_to_delete = ['name','nationality','club','position','height', 'weight','joined','loan date end','contract']

df_c.drop(columns=columns_to_delete, inplace=True)

df_c

,bp,foot,w/f,sm,a/w,d/w,ir
0,CB,Right,2 ★,2★,Low,High,1 ★
1,CAM,Right,4 ★,3★,High,Low,1 ★
2,GK,Right,2 ★,1★,Medium,Medium,1 ★
3,CDM,Right,3 ★,2★,Medium,Medium,1 ★
4,CDM,Right,4 ★,2★,Medium,Medium,1 ★
...,...,...,...,...,...,...,...
1994,LB,Left,3 ★,2★,High,Medium,1 ★
1995,GK,Right,2 ★,1★,Medium,Medium,1 ★
1996,RM,Left,4 ★,4★,High,Medium,2 ★
1997,CDM,Right,3 ★,2★,Medium,Medium,1 ★


In [28]:
df_c["bp"].value_counts()

CB     383
ST     289
CAM    261
GK     200
RM     164
CDM    153
CM     125
LB     103
LM      98
RB      91
RW      41
RWB     30
LWB     27
LW      23
CF      11
Name: bp, dtype: int64

In [29]:
df_c["ir"].value_counts()

1 ★    1759
2 ★     174
3 ★      54
4 ★      12
Name: ir, dtype: int64

In [30]:
df_c["sm"].value_counts()

2★    877
3★    776
1★    203
4★    136
5★      7
Name: sm, dtype: int64

In [31]:
df_c["w/f"].value_counts()

3 ★    1217
2 ★     405
4 ★     333
5 ★      36
1 ★       8
Name: w/f, dtype: int64

In [32]:
df_c["ir"].value_counts()

1 ★    1759
2 ★     174
3 ★      54
4 ★      12
Name: ir, dtype: int64

In [33]:
df_c.isna().sum()

bp      0
foot    0
w/f     0
sm      0
a/w     0
d/w     0
ir      0
dtype: int64

In [34]:
df_n=df.select_dtypes(include = np.number)

df_n.head()

,id,age,growth,value,wage,release clause,attacking,crossing,finishing,heading accuracy,short passing,volleys,skill,dribbling,curve,fk accuracy,long passing,ball control,movement,acceleration,sprint speed,agility,reactions,balance,power,shot power,jumping,stamina,strength,long shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing tackle,sliding tackle,goalkeeping,gk diving,gk handling,gk kicking,gk positioning,gk reflexes,total stats,base stats,pac,sho,pas,dri,def,phy,hits,ls,state,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,ova
0,219461,23,7,975000.0,5000.0,0.0,230,47,21,62,60,40.0,228,44,43.0,36,51,54,303,60,68,63.0,63,49.0,288,48,77.0,51,87,25,246,68,62.0,38.0,39.0,39,49.0,200,68,67,65.0,56,11,8,15,13,9,1551,334,64,30,50,50,66,74,34.0,48.0,48.0,48.0,48.0,47.0,47.0,47.0,48.0,48.0,48.0,48.0,50.0,51.0,51.0,51.0,50.0,59.0,61.0,61.0,61.0,59.0,61.0,67.0,67.0,67.0,61.0,16.0,67
1,221896,22,5,1200000.0,3000.0,2200000.0,298,62,60,44,62,70.0,330,76,68.0,56,60,70,375,77,72,83.0,64,79.0,323,76,62.0,63,47,75,286,72,26.0,64.0,64.0,60,65.0,61,19,23,19.0,53,14,13,9,9,8,1726,358,74,67,62,74,24,57,12.0,64.0,64.0,64.0,68.0,68.0,68.0,68.0,68.0,68.0,68.0,68.0,67.0,61.0,61.0,61.0,67.0,49.0,47.0,47.0,47.0,49.0,45.0,38.0,38.0,38.0,45.0,17.0,68
2,247428,19,17,120000.0,500.0,249000.0,48,7,5,11,21,4.0,52,6,8.0,8,20,10,165,28,25,33.0,41,38.0,171,40,49.0,22,54,6,76,20,9.0,7.0,26.0,14,31.0,27,8,9,10.0,269,56,52,53,53,55,808,295,56,52,53,55,26,53,3.0,18.0,18.0,18.0,15.0,17.0,17.0,17.0,15.0,17.0,17.0,17.0,16.0,18.0,18.0,18.0,16.0,16.0,18.0,18.0,18.0,16.0,16.0,18.0,18.0,18.0,16.0,53.0,54
3,255120,16,23,160000.0,500.0,464000.0,215,38,31,55,59,32.0,224,51,34.0,38,47,54,275,59,58,56.0,48,54.0,242,48,48.0,60,58,28,230,61,55.0,33.0,40.0,41,59.0,159,53,52,54.0,36,7,5,13,5,6,1381,303,58,34,47,52,53,59,6.0,46.0,46.0,46.0,47.0,46.0,46.0,46.0,47.0,47.0,47.0,47.0,49.0,49.0,49.0,49.0,49.0,53.0,54.0,54.0,54.0,53.0,53.0,54.0,54.0,54.0,53.0,11.0,55
4,215556,24,5,2300000.0,13000.0,4300000.0,295,57,59,45,78,56.0,327,71,57.0,51,74,74,320,68,66,66.0,64,56.0,337,73,56.0,74,72,62,314,66,78.0,53.0,62.0,55,63.0,211,72,68,71.0,60,12,7,13,15,13,1864,407,67,62,68,70,69,71,45.0,63.0,63.0,63.0,66.0,66.0,66.0,66.0,66.0,68.0,68.0,68.0,67.0,70.0,70.0,70.0,67.0,70.0,72.0,72.0,72.0,70.0,69.0,68.0,68.0,68.0,69.0,18.0,70


In [35]:
df_n=df_n.drop(['id'], axis=1)
df_n.head()

,age,growth,value,wage,release clause,attacking,crossing,finishing,heading accuracy,short passing,volleys,skill,dribbling,curve,fk accuracy,long passing,ball control,movement,acceleration,sprint speed,agility,reactions,balance,power,shot power,jumping,stamina,strength,long shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing tackle,sliding tackle,goalkeeping,gk diving,gk handling,gk kicking,gk positioning,gk reflexes,total stats,base stats,pac,sho,pas,dri,def,phy,hits,ls,state,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,ova
0,23,7,975000.0,5000.0,0.0,230,47,21,62,60,40.0,228,44,43.0,36,51,54,303,60,68,63.0,63,49.0,288,48,77.0,51,87,25,246,68,62.0,38.0,39.0,39,49.0,200,68,67,65.0,56,11,8,15,13,9,1551,334,64,30,50,50,66,74,34.0,48.0,48.0,48.0,48.0,47.0,47.0,47.0,48.0,48.0,48.0,48.0,50.0,51.0,51.0,51.0,50.0,59.0,61.0,61.0,61.0,59.0,61.0,67.0,67.0,67.0,61.0,16.0,67
1,22,5,1200000.0,3000.0,2200000.0,298,62,60,44,62,70.0,330,76,68.0,56,60,70,375,77,72,83.0,64,79.0,323,76,62.0,63,47,75,286,72,26.0,64.0,64.0,60,65.0,61,19,23,19.0,53,14,13,9,9,8,1726,358,74,67,62,74,24,57,12.0,64.0,64.0,64.0,68.0,68.0,68.0,68.0,68.0,68.0,68.0,68.0,67.0,61.0,61.0,61.0,67.0,49.0,47.0,47.0,47.0,49.0,45.0,38.0,38.0,38.0,45.0,17.0,68
2,19,17,120000.0,500.0,249000.0,48,7,5,11,21,4.0,52,6,8.0,8,20,10,165,28,25,33.0,41,38.0,171,40,49.0,22,54,6,76,20,9.0,7.0,26.0,14,31.0,27,8,9,10.0,269,56,52,53,53,55,808,295,56,52,53,55,26,53,3.0,18.0,18.0,18.0,15.0,17.0,17.0,17.0,15.0,17.0,17.0,17.0,16.0,18.0,18.0,18.0,16.0,16.0,18.0,18.0,18.0,16.0,16.0,18.0,18.0,18.0,16.0,53.0,54
3,16,23,160000.0,500.0,464000.0,215,38,31,55,59,32.0,224,51,34.0,38,47,54,275,59,58,56.0,48,54.0,242,48,48.0,60,58,28,230,61,55.0,33.0,40.0,41,59.0,159,53,52,54.0,36,7,5,13,5,6,1381,303,58,34,47,52,53,59,6.0,46.0,46.0,46.0,47.0,46.0,46.0,46.0,47.0,47.0,47.0,47.0,49.0,49.0,49.0,49.0,49.0,53.0,54.0,54.0,54.0,53.0,53.0,54.0,54.0,54.0,53.0,11.0,55
4,24,5,2300000.0,13000.0,4300000.0,295,57,59,45,78,56.0,327,71,57.0,51,74,74,320,68,66,66.0,64,56.0,337,73,56.0,74,72,62,314,66,78.0,53.0,62.0,55,63.0,211,72,68,71.0,60,12,7,13,15,13,1864,407,67,62,68,70,69,71,45.0,63.0,63.0,63.0,66.0,66.0,66.0,66.0,66.0,68.0,68.0,68.0,67.0,70.0,70.0,70.0,67.0,70.0,72.0,72.0,72.0,70.0,69.0,68.0,68.0,68.0,69.0,18.0,70


In [36]:
column_means = df_n.mean()

# Fill NaN values with column means
df_n.fillna(column_means, inplace=True)
df_n

,age,growth,value,wage,release clause,attacking,crossing,finishing,heading accuracy,short passing,volleys,skill,dribbling,curve,fk accuracy,long passing,ball control,movement,acceleration,sprint speed,agility,reactions,balance,power,shot power,jumping,stamina,strength,long shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing tackle,sliding tackle,goalkeeping,gk diving,gk handling,gk kicking,gk positioning,gk reflexes,total stats,base stats,pac,sho,pas,dri,def,phy,hits,ls,state,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,ova
0,23,7,975000.0,5000.0,0.0,230,47,21,62,60,40.0,228,44,43.0,36,51,54,303,60,68,63.0,63,49.0,288,48,77.0,51,87,25,246,68,62.0,38.0,39.0,39,49.0,200,68,67,65.0,56,11,8,15,13,9,1551,334,64,30,50,50,66,74,34.0,48.0,48.0,48.0,48.0,47.0,47.0,47.0,48.0,48.0,48.0,48.0,50.0,51.0,51.0,51.0,50.0,59.0,61.0,61.0,61.0,59.0,61.0,67.0,67.0,67.0,61.0,16.0,67
1,22,5,1200000.0,3000.0,2200000.0,298,62,60,44,62,70.0,330,76,68.0,56,60,70,375,77,72,83.0,64,79.0,323,76,62.0,63,47,75,286,72,26.0,64.0,64.0,60,65.0,61,19,23,19.0,53,14,13,9,9,8,1726,358,74,67,62,74,24,57,12.0,64.0,64.0,64.0,68.0,68.0,68.0,68.0,68.0,68.0,68.0,68.0,67.0,61.0,61.0,61.0,67.0,49.0,47.0,47.0,47.0,49.0,45.0,38.0,38.0,38.0,45.0,17.0,68
2,19,17,120000.0,500.0,249000.0,48,7,5,11,21,4.0,52,6,8.0,8,20,10,165,28,25,33.0,41,38.0,171,40,49.0,22,54,6,76,20,9.0,7.0,26.0,14,31.0,27,8,9,10.0,269,56,52,53,53,55,808,295,56,52,53,55,26,53,3.0,18.0,18.0,18.0,15.0,17.0,17.0,17.0,15.0,17.0,17.0,17.0,16.0,18.0,18.0,18.0,16.0,16.0,18.0,18.0,18.0,16.0,16.0,18.0,18.0,18.0,16.0,53.0,54
3,16,23,160000.0,500.0,464000.0,215,38,31,55,59,32.0,224,51,34.0,38,47,54,275,59,58,56.0,48,54.0,242,48,48.0,60,58,28,230,61,55.0,33.0,40.0,41,59.0,159,53,52,54.0,36,7,5,13,5,6,1381,303,58,34,47,52,53,59,6.0,46.0,46.0,46.0,47.0,46.0,46.0,46.0,47.0,47.0,47.0,47.0,49.0,49.0,49.0,49.0,49.0,53.0,54.0,54.0,54.0,53.0,53.0,54.0,54.0,54.0,53.0,11.0,55
4,24,5,2300000.0,13000.0,4300000.0,295,57,59,45,78,56.0,327,71,57.0,51,74,74,320,68,66,66.0,64,56.0,337,73,56.0,74,72,62,314,66,78.0,53.0,62.0,55,63.0,211,72,68,71.0,60,12,7,13,15,13,1864,407,67,62,68,70,69,71,45.0,63.0,63.0,63.0,66.0,66.0,66.0,66.0,66.0,68.0,68.0,68.0,67.0,70.0,70.0,70.0,67.0,70.0,72.0,72.0,72.0,70.0,69.0,68.0,68.0,68.0,69.0,18.0,70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1994,21,11,325000.0,1000.0,731000.0,197,59,23,42,51,22.0,207,51,30.0,29,41,56,344,74,78,63.0,50,79.0,245,31,66.0,68,50,30,232,59,52.0,51.0,34.0,36,59.0,167,52,57,58.0,51,6,15,7,10,13,1443,315,76,28,46,55,53,57,4.0,44.0,44.0,44.0,51.0,48.0,48.0,48.0,51.0,48.0,48.0,48.0,53.0,48.0,48.0,48.0,53.0,58.0,52.0,52.0,52.0,58.0,58.0,54.0,54.0,54.0,58.0,15.0,60
1995,21,9,190000.0,700.0,285000.0,77,13,9,14,34,7.0,70,9,13.0,10,25,13,192,31,30,36.0,59,36.0,177,43,44.0,23,60,7,88,21,8.0,4.0,36.0,19,33.0,36,14,11,11.0,288,60,55,57,54,62,928,318,60,55,57,62,30,54,3.0,22.0,22.0,22.0,20.0,22.0,22.0,22.0,20.0,23.0,23.0,23.0,22.0,24.0,24.0,24.0,22.0,20.0,23.0,23.0,23.0,20.0,20.0,22.0,22.0,22.0,20.0,58.0,59
1996,27,0,8000000.0,9000.0,12000000.0,332,76,72,34,79,71.0,373,77,76.0,65,77,78,351,64,65,74.0,73,75.0,329,63,60.0,83,57,66,306,41,40.0,75.0,81.0,69,75.0,118,36,43,39.0,58,12,15,16,9,6,1867,388,65,69,78,77,39,60,15.0,67.0,67.0,67.0,74.0,73.0,73.0,73.0,74.0,76.0,76.0,76.0,75.0,74.0,74.0,74.0,75.0,62.0,60.0,60.0,60.0,62.0,57.0,47.0,47.0,47.0,57.0,18.0,76
1997,34,0,140000.0,4000.0,245000.0,268,58,44,61,62,43.0,261,56,47.0,37,60,61,288,54,52,57.0,61,64.0,300,53,65.0,68,68,46,297,74,62.0,54.0,56.0,51,60.0,182,61,62,59.0,43,10,7,9,9,8,1639,346,53,47,58,58,61,69,4.0,55.0,55.0,55.0,56.0,55.0,55.0,55.0,56.0,57.0,57.0,

In [37]:
df=pd.concat([df_c,df_n], axis=1)
df

,bp,foot,w/f,sm,a/w,d/w,ir,age,growth,value,wage,release clause,attacking,crossing,finishing,heading accuracy,short passing,volleys,skill,dribbling,curve,fk accuracy,long passing,ball control,movement,acceleration,sprint speed,agility,reactions,balance,power,shot power,jumping,stamina,strength,long shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing tackle,sliding tackle,goalkeeping,gk diving,gk handling,gk kicking,gk positioning,gk reflexes,total stats,base stats,pac,sho,pas,dri,def,phy,hits,ls,state,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,ova
0,CB,Right,2 ★,2★,Low,High,1 ★,23,7,975000.0,5000.0,0.0,230,47,21,62,60,40.0,228,44,43.0,36,51,54,303,60,68,63.0,63,49.0,288,48,77.0,51,87,25,246,68,62.0,38.0,39.0,39,49.0,200,68,67,65.0,56,11,8,15,13,9,1551,334,64,30,50,50,66,74,34.0,48.0,48.0,48.0,48.0,47.0,47.0,47.0,48.0,48.0,48.0,48.0,50.0,51.0,51.0,51.0,50.0,59.0,61.0,61.0,61.0,59.0,61.0,67.0,67.0,67.0,61.0,16.0,67
1,CAM,Right,4 ★,3★,High,Low,1 ★,22,5,1200000.0,3000.0,2200000.0,298,62,60,44,62,70.0,330,76,68.0,56,60,70,375,77,72,83.0,64,79.0,323,76,62.0,63,47,75,286,72,26.0,64.0,64.0,60,65.0,61,19,23,19.0,53,14,13,9,9,8,1726,358,74,67,62,74,24,57,12.0,64.0,64.0,64.0,68.0,68.0,68.0,68.0,68.0,68.0,68.0,68.0,67.0,61.0,61.0,61.0,67.0,49.0,47.0,47.0,47.0,49.0,45.0,38.0,38.0,38.0,45.0,17.0,68
2,GK,Right,2 ★,1★,Medium,Medium,1 ★,19,17,120000.0,500.0,249000.0,48,7,5,11,21,4.0,52,6,8.0,8,20,10,165,28,25,33.0,41,38.0,171,40,49.0,22,54,6,76,20,9.0,7.0,26.0,14,31.0,27,8,9,10.0,269,56,52,53,53,55,808,295,56,52,53,55,26,53,3.0,18.0,18.0,18.0,15.0,17.0,17.0,17.0,15.0,17.0,17.0,17.0,16.0,18.0,18.0,18.0,16.0,16.0,18.0,18.0,18.0,16.0,16.0,18.0,18.0,18.0,16.0,53.0,54
3,CDM,Right,3 ★,2★,Medium,Medium,1 ★,16,23,160000.0,500.0,464000.0,215,38,31,55,59,32.0,224,51,34.0,38,47,54,275,59,58,56.0,48,54.0,242,48,48.0,60,58,28,230,61,55.0,33.0,40.0,41,59.0,159,53,52,54.0,36,7,5,13,5,6,1381,303,58,34,47,52,53,59,6.0,46.0,46.0,46.0,47.0,46.0,46.0,46.0,47.0,47.0,47.0,47.0,49.0,49.0,49.0,49.0,49.0,53.0,54.0,54.0,54.0,53.0,53.0,54.0,54.0,54.0,53.0,11.0,55
4,CDM,Right,4 ★,2★,Medium,Medium,1 ★,24,5,2300000.0,13000.0,4300000.0,295,57,59,45,78,56.0,327,71,57.0,51,74,74,320,68,66,66.0,64,56.0,337,73,56.0,74,72,62,314,66,78.0,53.0,62.0,55,63.0,211,72,68,71.0,60,12,7,13,15,13,1864,407,67,62,68,70,69,71,45.0,63.0,63.0,63.0,66.0,66.0,66.0,66.0,66.0,68.0,68.0,68.0,67.0,70.0,70.0,70.0,67.0,70.0,72.0,72.0,72.0,70.0,69.0,68.0,68.0,68.0,69.0,18.0,70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1994,LB,Left,3 ★,2★,High,Medium,1 ★,21,11,325000.0,1000.0,731000.0,197,59,23,42,51,22.0,207,51,30.0,29,41,56,344,74,78,63.0,50,79.0,245,31,66.0,68,50,30,232,59,52.0,51.0,34.0,36,59.0,167,52,57,58.0,51,6,15,7,10,13,1443,315,76,28,46,55,53,57,4.0,44.0,44.0,44.0,51.0,48.0,48.0,48.0,51.0,48.0,48.0,48.0,53.0,48.0,48.0,48.0,53.0,58.0,52.0,52.0,52.0,58.0,58.0,54.0,54.0,54.0,58.0,15.0,60
1995,GK,Right,2 ★,1★,Medium,Medium,1 ★,21,9,190000.0,700.0,285000.0,77,13,9,14,34,7.0,70,9,13.0,10,25,13,192,31,30,36.0,59,36.0,177,43,44.0,23,60,7,88,21,8.0,4.0,36.0,19,33.0,36,14,11,11.0,288,60,55,57,54,62,928,318,60,55,57,62,30,54,3.0,22.0,22.0,22.0,20.0,22.0,22.0,22.0,20.0,23.0,23.0,23.0,22.0,24.0,24.0,24.0,22.0,20.0,23.0,23.0,23.0,20.0,20.0,22.0,22.0,22.0,20.0,58.0,59
1996,RM,Left,4 ★,4★,High,Medium,2 ★,27,0,8000000.0,9000.0,12000000.0,332,76,72,34,79,71.0,373,77,76.0,65,77,78,351,64,65,74.0,73,75.0,329,63,60.0,83,57,66,306,41,40.0,75.0,81.0,69,75.0,118,36,43,39.0,58,12,15,16,9,6,1867,388,65,69,78,77,39,60,15.0,67.0,67.0,67.0,74.0,73.0,73.0,73.0,74.0,76.0,76.0,76.0,75.0,74.0,74.0,74.0,75.0,62.0,6

In [38]:
df_n.isna().sum()

age               0
growth            0
value             0
wage              0
release clause    0
                 ..
cb                0
rcb               0
rb                0
gk                0
ova               0
Length: 83, dtype: int64

## Normalization

In [39]:
#separate the features from the labels
y = df['ova']
X = df.drop(['ova'], axis=1)

In [40]:
#categorical features and numerical ones are going to be treated differently
X_num = X.select_dtypes(include = np.number)
X_cat = X.select_dtypes(include = object)

In [41]:
X_num.describe().T

,count,mean,std,min,25%,50%,75%,max
age,1999.0,2.537719e+01,4.867082e+00,16.0,21.5,25.0,29.0,47.0
growth,1999.0,5.365183e+00,5.715451e+00,0.0,0.0,4.0,9.0,25.0
value,1999.0,2.583663e+06,5.451206e+06,0.0,375000.0,800000.0,2400000.0,78000000.0
wage,1999.0,1.048554e+04,2.008816e+04,0.0,1000.0,3000.0,10000.0,250000.0
release clause,1999.0,4.679741e+06,1.076902e+07,0.0,544000.0,1300000.0,4000000.0,159400000.0
...,...,...,...,...,...,...,...,...
lcb,1999.0,5.337069e+01,1.476846e+01,17.0,43.0,56.0,65.0,86.0
cb,1999.0,5.337069e+01,1.476846e+01,17.0,43.0,56.0,65.0,86.0
rcb,1999.0,5.337069e+01,1.476846e+01,17.0,43.0,56.0,65.0,86.0
rb,1999.0,5.446373e+01,1.375463e+01,15.0,48.0,57.0,64.0,82.0


In [42]:
from sklearn.preprocessing import MinMaxScaler
MinMaxtransformer = MinMaxScaler().fit(X_num)
X_normalized = MinMaxtransformer.transform(X_num)
print(type(X_normalized))
X_normalized = pd.DataFrame(X_normalized,columns=X_num.columns)
display(X_normalized.head())
print(type(X_normalized))

<class 'numpy.ndarray'>


,age,growth,value,wage,release clause,attacking,crossing,finishing,heading accuracy,short passing,volleys,skill,dribbling,curve,fk accuracy,long passing,ball control,movement,acceleration,sprint speed,agility,reactions,balance,power,shot power,jumping,stamina,strength,long shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing tackle,sliding tackle,goalkeeping,gk diving,gk handling,gk kicking,gk positioning,gk reflexes,total stats,base stats,pac,sho,pas,dri,def,phy,hits,ls,state,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
0,0.225806,0.28,0.012500,0.020,0.000000,0.509804,0.493976,0.195402,0.635294,0.604938,0.444444,0.463158,0.448276,0.430233,0.337209,0.5125,0.543210,0.535503,0.578313,0.679012,0.592105,0.524590,0.386667,0.541219,0.418919,0.774648,0.481928,0.885714,0.247059,0.517906,0.690476,0.690476,0.390805,0.358025,0.376471,0.4625,0.708502,0.707865,0.719512,0.695122,0.097156,0.095238,0.066667,0.152174,0.125000,0.079545,0.526038,0.338710,0.531250,0.138889,0.328125,0.327869,0.688312,0.719298,0.033369,0.455882,0.455882,0.455882,0.459459,0.438356,0.438356,0.438356,0.459459,0.452055,0.452055,0.452055,0.486111,0.521739,0.521739,0.521739,0.486111,0.647059,0.647059,0.647059,0.647059,0.647059,0.686567,0.724638,0.724638,0.724638,0.686567,0.0875
1,0.193548,0.20,0.015385,0.012,0.013802,0.700280,0.674699,0.643678,0.423529,0.629630,0.814815,0.731579,0.816092,0.720930,0.569767,0.6250,0.740741,0.748521,0.783133,0.728395,0.855263,0.540984,0.786667,0.666667,0.797297,0.563380,0.626506,0.314286,0.835294,0.628099,0.738095,0.261905,0.689655,0.666667,0.623529,0.6625,0.145749,0.157303,0.182927,0.134146,0.090047,0.130952,0.122222,0.086957,0.079545,0.068182,0.641397,0.435484,0.687500,0.652778,0.515625,0.721311,0.142857,0.421053,0.009688,0.691176,0.691176,0.691176,0.729730,0.726027,0.726027,0.726027,0.729730,0.726027,0.726027,0.726027,0.722222,0.666667,0.666667,0.666667,0.722222,0.500000,0.441176,0.441176,0.441176,0.500000,0.447761,0.304348,0.304348,0.304348,0.447761,0.1000
2,0.096774,0.68,0.001538,0.002,0.001562,0.000000,0.012048,0.011494,0.035294,0.123457,0.000000,0.000000,0.011494,0.023256,0.011628,0.1250,0.000000,0.127219,0.192771,0.148148,0.197368,0.163934,0.240000,0.121864,0.310811,0.380282,0.132530,0.414286,0.023529,0.049587,0.119048,0.059524,0.034483,0.197531,0.082353,0.2375,0.008097,0.033708,0.012195,0.024390,0.601896,0.630952,0.555556,0.565217,0.579545,0.602273,0.036256,0.181452,0.406250,0.444444,0.375000,0.409836,0.168831,0.350877,0.000000,0.014706,0.014706,0.014706,0.013514,0.027397,0.027397,0.027397,0.013514,0.027397,0.027397,0.027397,0.013889,0.043478,0.043478,0.043478,0.013889,0.014706,0.014706,0.014706,0.014706,0.014706,0.014925,0.014493,0.014493,0.014493,0.014925,0.5500
3,0.000000,0.92,0.002051,0.002,0.002911,0.467787,0.385542,0.310345,0.552941,0.592593,0.345679,0.452632,0.528736,0.325581,0.360465,0.4625,0.543210,0.452663,0.566265,0.555556,0.500000,0.278689,0.453333,0.376344,0.418919,0.366197,0.590361,0.471429,0.282353,0.473829,0.607143,0.607143,0.333333,0.370370,0.400000,0.5875,0.542510,0.539326,0.536585,0.560976,0.049763,0.047619,0.033333,0.130435,0.034091,0.045455,0.413975,0.213710,0.437500,0.194444,0.281250,0.360656,0.519481,0.456140,0.003229,0.426471,0.426471,0.426471,0.445946,0.424658,0.424658,0.424658,0.445946,0.438356,0.438356,0.438356,0.472222,0.492754,0.492754,0.492754,0.472222,0.558824,0.544118,0.544118,0.544118,0.558824,0.567164,0.536232,0.536232,0.536232,0.567164,0.0250
4,0.258065,0.20,0.029487,0.052,0.026976,0.691877,0.614458,0.632184,0.435294,0.827160,0.641975,0.723684,0.758621,0.593023,0.511628,0.8000,0.790123,0.585799,0.674699,0.654321,0.631579,0.540984,0.480000,0.716846,0.756757,0.478873,0.759036,0.671429,0.682353,0.705234,0.666667,0.880952,0.563218,0.641975,0.564706,0.6375,0.753036,0.752809,0.731707,0.768293,0.106635,0.107143,0.055556,0.130435,0.147727,0.125000,0.732367,0.633065,0.578125,0.583333,0.609375,0.655738,0.727273,0

<class 'pandas.core.frame.DataFrame'>


In [43]:
pickle.dump(MinMaxtransformer, open('scalar.pkl', 'wb'))

In [44]:
# # plots before normalising
# sns.set(style="whitegrid")
# for column in df_n:
#     plt.figure(figsize=(6, 4))
#     sns.histplot(data=df_n, x=column, kde=True)
#     plt.title(f'Distribution of {column}')
#     plt.show()

In [45]:
# # plots after normalising
# sns.set(style="whitegrid")
# for column in X_normalized:
#     plt.figure(figsize=(6, 4))
#     sns.histplot(data=X_normalized, x=column, kde=True)
#     plt.title(f'Distribution of {column}')
#     plt.show()

# One Hot Encoding (categorical).


In [46]:
X_cat = df.select_dtypes(include = object)

In [47]:
#one hot encoding is a way to turn categorical variables into multiple numerical columns
from sklearn.preprocessing import OneHotEncoder
# encoder = OneHotEncoder().fit(X_cat)
encoder = OneHotEncoder(drop='first').fit(X_cat)
print(encoder.categories_)
encoded = encoder.transform(X_cat).toarray()
print(encoded)

[array(['CAM', 'CB', 'CDM', 'CF', 'CM', 'GK', 'LB', 'LM', 'LW', 'LWB',
       'RB', 'RM', 'RW', 'RWB', 'ST'], dtype=object), array(['Left', 'Right'], dtype=object), array(['1 ★', '2 ★', '3 ★', '4 ★', '5 ★'], dtype=object), array(['1★', '2★', '3★', '4★', '5★'], dtype=object), array(['High', 'Low', 'Medium'], dtype=object), array(['High', 'Low', 'Medium'], dtype=object), array(['1 ★', '2 ★', '3 ★', '4 ★'], dtype=object)]
[[1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [48]:
encoder = OneHotEncoder(drop='first').fit(X_cat)

cols = encoder.get_feature_names_out(input_features=X_cat.columns)

X_cat_encode = pd.DataFrame(encoder.transform(X_cat).toarray(),columns=cols)

X_cat_encode.head()

,bp_CB,bp_CDM,bp_CF,bp_CM,bp_GK,bp_LB,bp_LM,bp_LW,bp_LWB,bp_RB,bp_RM,bp_RW,bp_RWB,bp_ST,foot_Right,w/f_2 ★,w/f_3 ★,w/f_4 ★,w/f_5 ★,sm_2★,sm_3★,sm_4★,sm_5★,a/w_Low,a/w_Medium,d/w_Low,d/w_Medium,ir_2 ★,ir_3 ★,ir_4 ★
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0


In [49]:
pickle.dump(encoder, open('encoder.pkl', 'wb'))

In [50]:
# concatinating the normalised and encoded data
X = pd.concat([X_normalized,X_cat_encode], axis=1)  # np.concatenate()
X

,age,growth,value,wage,release clause,attacking,crossing,finishing,heading accuracy,short passing,volleys,skill,dribbling,curve,fk accuracy,long passing,ball control,movement,acceleration,sprint speed,agility,reactions,balance,power,shot power,jumping,stamina,strength,long shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing tackle,sliding tackle,goalkeeping,gk diving,gk handling,gk kicking,gk positioning,gk reflexes,total stats,base stats,pac,sho,pas,dri,def,phy,hits,ls,state,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,bp_CB,bp_CDM,bp_CF,bp_CM,bp_GK,bp_LB,bp_LM,bp_LW,bp_LWB,bp_RB,bp_RM,bp_RW,bp_RWB,bp_ST,foot_Right,w/f_2 ★,w/f_3 ★,w/f_4 ★,w/f_5 ★,sm_2★,sm_3★,sm_4★,sm_5★,a/w_Low,a/w_Medium,d/w_Low,d/w_Medium,ir_2 ★,ir_3 ★,ir_4 ★
0,0.225806,0.28,0.012500,0.0200,0.000000,0.509804,0.493976,0.195402,0.635294,0.604938,0.444444,0.463158,0.448276,0.430233,0.337209,0.5125,0.543210,0.535503,0.578313,0.679012,0.592105,0.524590,0.386667,0.541219,0.418919,0.774648,0.481928,0.885714,0.247059,0.517906,0.690476,0.690476,0.390805,0.358025,0.376471,0.4625,0.708502,0.707865,0.719512,0.695122,0.097156,0.095238,0.066667,0.152174,0.125000,0.079545,0.526038,0.338710,0.531250,0.138889,0.328125,0.327869,0.688312,0.719298,0.033369,0.455882,0.455882,0.455882,0.459459,0.438356,0.438356,0.438356,0.459459,0.452055,0.452055,0.452055,0.486111,0.521739,0.521739,0.521739,0.486111,0.647059,0.647059,0.647059,0.647059,0.647059,0.686567,0.724638,0.724638,0.724638,0.686567,0.0875,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.193548,0.20,0.015385,0.0120,0.013802,0.700280,0.674699,0.643678,0.423529,0.629630,0.814815,0.731579,0.816092,0.720930,0.569767,0.6250,0.740741,0.748521,0.783133,0.728395,0.855263,0.540984,0.786667,0.666667,0.797297,0.563380,0.626506,0.314286,0.835294,0.628099,0.738095,0.261905,0.689655,0.666667,0.623529,0.6625,0.145749,0.157303,0.182927,0.134146,0.090047,0.130952,0.122222,0.086957,0.079545,0.068182,0.641397,0.435484,0.687500,0.652778,0.515625,0.721311,0.142857,0.421053,0.009688,0.691176,0.691176,0.691176,0.729730,0.726027,0.726027,0.726027,0.729730,0.726027,0.726027,0.726027,0.722222,0.666667,0.666667,0.666667,0.722222,0.500000,0.441176,0.441176,0.441176,0.500000,0.447761,0.304348,0.304348,0.304348,0.447761,0.1000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.096774,0.68,0.001538,0.0020,0.001562,0.000000,0.012048,0.011494,0.035294,0.123457,0.000000,0.000000,0.011494,0.023256,0.011628,0.1250,0.000000,0.127219,0.192771,0.148148,0.197368,0.163934,0.240000,0.121864,0.310811,0.380282,0.132530,0.414286,0.023529,0.049587,0.119048,0.059524,0.034483,0.197531,0.082353,0.2375,0.008097,0.033708,0.012195,0.024390,0.601896,0.630952,0.555556,0.565217,0.579545,0.602273,0.036256,0.181452,0.406250,0.444444,0.375000,0.409836,0.168831,0.350877,0.000000,0.014706,0.014706,0.014706,0.013514,0.027397,0.027397,0.027397,0.013514,0.027397,0.027397,0.027397,0.013889,0.043478,0.043478,0.043478,0.013889,0.014706,0.014706,0.014706,0.014706,0.014706,0.014925,0.014493,0.014493,0.014493,0.014925,0.5500,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
3,0.000000,0.92,0.002051,0.0020,0.002911,0.467787,0.385542,0.310345,0.552941,0.592593,0.345679,0.452632,0.528736,0.325581,0.360465,0.4625,0.543210,0.452663,0.566265,0.555556,0.500000,0.278689,0.453333,0.376344,0.418919,0.366197,0.590361,0.471429,0.282353,0.473829,0.607143,0.607143,0.333333,0.370370,0.400000,0.5875,0.542510,0.539326,0.536585,0.560976,0.049763,0.047619,0.033333,0.130435,0.034091,0.045455,0.413975,0.213710,0.437500,0.194444,0.281250,0.360656,0.519481,0.456140,0.003229,0.426471,0.426471,0.426471,0.445946,0.424658,0.424658,0.424658,0.445946,0.438356,0.438356,0.438356,0.472222,0.492754,0.492754,0.49275

In [51]:
# X[""].describe().T

# Splitting into train set and test set.

In [52]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [53]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1599, 112)
(400, 112)
(1599,)
(400,)


# Modeling

In [54]:
# Applying linear regression model
lm = linear_model.LinearRegression()
lm.fit(X_train,y_train)

LinearRegression()

In [55]:
pickle.dump(lm, open('model.pkl', 'wb'))

# Model Validation

In [56]:
from sklearn.metrics import mean_absolute_error, r2_score

In [57]:
lm.predict(X_train)

array([63.35644531, 64.75830078, 62.41308594, ..., 63.06152344,
       67.953125  , 69.89550781])

In [58]:
# r2 score
predictions = lm.predict(X_train)
r2_score(y_train, predictions)

0.9172191587283648

In [59]:
# r2 score
predictions1 = lm.predict(X_test)
r2_score(y_test, predictions1)

0.8976671863894781

In [60]:
from sklearn.metrics import mean_squared_error

# RMSE
rmse = np.sqrt(mean_squared_error(y_test,predictions1))
rmse

2.1371500266937415

In [61]:
#MAE 
mae = mean_absolute_error(y_test, predictions1)
print(mae)

1.66244384765625


In [62]:
MSE = mean_squared_error(y_train,predictions)
MSE.round(2)

3.81